In [ ]:
from qdrant_client import QdrantClient, models

client = QdrantClient(host="localhost", port=6333)

In [ ]:
from fastembed import TextEmbedding

dense_model = TextEmbedding(model_name="jinaai/jina-embeddings-v3", 
                            providers=["CUDAExecutionProvider"])

def batch_embeddings_dense(contents: list[str]):
    all_embeddings = []
    
    for i in range(0, len(contents), 100):
        batch = contents[i:i+100]
        embeddings = list(dense_model.query_embed(batch))
        all_embeddings.extend(embeddings)
    
    return all_embeddings

In [ ]:
from fastembed import SparseTextEmbedding

SparseTextEmbedding.list_supported_models()

sparse_model = SparseTextEmbedding(model_name="prithivida/Splade_PP_en_v1", 
                                   providers=["CUDAExecutionProvider"])

def batch_embeddings_sparse(contents: list[str]):
    all_embeddings = []
    
    for i in range(0, len(contents), 100):
        batch = contents[i:i+100]
        embeddings = list(sparse_model.query_embed(batch))
        all_embeddings.extend(embeddings)
    
    return all_embeddings


In [ ]:
import json

queries = [

]

for i, query in enumerate(queries):
    result = client.query_points(
        collection_name="docs360",
        prefetch=[
            models.Prefetch(query=batch_embeddings_dense([query])[0], using="dense", limit=50),
            models.Prefetch(query=batch_embeddings_sparse([query])[0].as_object(), using="splade", limit=50),
        ],
        query=models.FusionQuery(fusion=models.Fusion.DBSF),
        limit=20
    )


    final = {
        "query": query,
        "result": result.model_dump()["points"],
    }

    w = open(f"queries/{query}.json", "w")

    w.write(json.dumps(final, indent=2))

    w.close()

